# Scraping dynamic web sites with Selenium

[Last week's lesson](../pt1/scraping_lecture.ipynb) involved scraping a static site, or a site that is rendered up front in HTML. Today, we'll look at how to scrape sites that change when you load or interact with the page, sometimes without the URL changing.

[Selenium](https://www.selenium.dev/documentation/) was created to "automate browsers." The major use case for software like Selenium is to automate testing browser-based apps. But journalists can use software like Selenium to scrape dynamic websites.

For today's lesson, we're going to scrape all the public hearings in Alameda County courts on a given day.

In [1]:
!pip install selenium

In [2]:
# see the versions
!pip freeze

anyio==3.4.0
appnope==0.1.2
argon2-cffi==21.1.0
async-generator==1.10
attrs==21.2.0
Babel==2.9.1
backcall==0.2.0
beautifulsoup4==4.10.0
bleach==4.1.0
bs4==0.0.1
certifi==2021.10.8
cffi==1.15.0
charset-normalizer==2.0.8
cryptography==36.0.0
debugpy==1.5.1
decorator==5.1.0
defusedxml==0.7.1
entrypoints==0.3
h11==0.12.0
idna==3.3
importlib-resources==5.4.0
ipykernel==6.5.1
ipython==7.30.0
ipython-genutils==0.2.0
ipywidgets==7.6.5
jedi==0.18.1
Jinja2==3.0.3
json5==0.9.6
jsonschema==4.2.1
jupyter==1.0.0
jupyter-client==7.1.0
jupyter-console==6.4.0
jupyter-contrib-core==0.3.3
jupyter-core==4.9.1
jupyter-nbextensions-configurator==0.4.1
jupyter-server==1.12.1
jupyterlab==3.2.4
jupyterlab-pygments==0.1.2
jupyterlab-server==2.8.2
jupyterlab-widgets==1.0.2
MarkupSafe==2.0.1
matplotlib-inline==0.1.3
mistune==0.8.4
nbclassic==0.3.4
nbclient==0.5.9
nbconvert==6.3.0
nbformat==5.1.3
nest-asyncio==1.5.1
notebook==6.4.6
numpy==1.21.4
outcome==1.1.0
packaging==21.3
pandas==1.3.4
pandocfilters==1.5.0
par

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By

import time
import math

/Users/KaylaDong/.pyenv/versions/3.8.5/envs/scraping-lecture-3.8.5/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Open your automated browser

Earlier we installed `chromedriver` using `brew`. Below, we tell Selenium to use Chrome as our automated browser.

In [4]:
# initiate webdriver
driver = webdriver.Chrome()

# some people like to call this variable `browser` — call it whatever you like!

## Open the website

In [5]:
# Navigate to a URL
driver.get('https://publicrecords.alameda.courts.ca.gov/CalendarSearch/')

## Find the inputs you want to interact with

In last week's lecture, we used Beautiful Soup to find elements on a page. Because we want to interact with elements within Selenium's automated browser, we need to use Selenium to find elements. 

Tips:
- If you want to interact with the page, use selenium
- If you want to read or parse complex HTML, use bs4

You'll use `By` to indicate how the browser will pinpoint your element. These are the [different options for `By`](https://www.selenium.dev/selenium/docs/api/py/webdriver/selenium.webdriver.common.by.html):

- `CLASS_NAME`
- `CSS_SELECTOR` (e.g. a pseudo-element)
- `ID`
- `LINK_TEXT` (the text inside <a> tags)
- `NAME`
- `PARTIAL_LINK_TEXT` (the text inside <a> tags)
- `TAG_NAME`
- `XPATH` (when the element doesn't have a unique identifer, you can still pinpoint with this method; Chrome has a cool way to grab the xpath of an item in Developer Tools)

Luckily, the date fields have IDs, so we can select them this way:

In [6]:
hearing_date_from = driver.find_element(By.ID, 'FeaturedContent_txtFromdt')
hearing_date_to   = driver.find_element(By.ID, 'FeaturedContent_txtTodt')

You can use `type()` to find out whether a variable is a selenium object or a bs4 object.

In [7]:
type(hearing_date_from)

selenium.webdriver.remote.webelement.WebElement

## Input dates into the dropdowns

Use selenium's `send_keys()` method to input text into the date dropdowns.

In [8]:
hearing_date_from.send_keys('12/06/2021')
hearing_date_to.send_keys('12/06/2021')

## "Click" on the submit button

First, you'll have to find the element by its `id` value, then `click()` on it.

In [9]:
submit_button = driver.find_element(By.ID, 'FeaturedContent_btFind')

In [10]:
submit_button.click()

Below, I'm telling the computer to wait 5 seconds before executing the next line of code. That way the browser can finish loading the page before continuing with the code. That's crucial if I end up restarting this notebook kernel and running all cells at once. We want the browser to finish loading the page because some elements might not exist until the element exists. 

In [11]:
time.sleep(5)

There are better ways to wait for elements on a page. Check out the documentation to read more about [WebDriverWait()](https://selenium-python.readthedocs.io/waits.html).

## "Select" more rows to view

When you  get your search results, the courts show only 10 rows at a time. It'll be faster to scrape all the results if you can show the max amount of rows at a time (which is 50).

In [12]:
displayed_rows_dropdown = Select(driver.find_element(By.NAME, 'ctl00$MainContent$gvResult$ctl13$ctl13'))

In [13]:
displayed_rows_dropdown.select_by_visible_text('50')

## Get the count of results so you know how many pages you have to scrape

Even though I'm parsing HTML below, I'm using Selenium instead of Beautiful Soup. I'm doing this because I haven't called Beautiful Soup yet and Selenium is capable of parsing.

In [14]:
records_count_container = driver.find_element(By.ID, 'MainContent_lbCnt')
records_count = records_count_container.text.split()
records_count = records_count[len(records_count) - 1]
records_count = int(records_count)
records_count

1395

In [15]:
pages_to_check = math.ceil(records_count/50)
pages_to_check

28

## Figure out how to loop through the pages

In [16]:
# find the "Next" link — it looks like ">"
next_button = driver.find_element(By.LINK_TEXT, '>')
next_button.click()

The below code is commented out because I don't want you to run it yet. But, you can see how one could flip through all the pages of this site.

In [17]:
# for n in range(pages_to_check):
#     next_button = driver.find_element(By.LINK_TEXT, '>')
#     next_button.click()
    
#     # wait 2 seconds
#     time.sleep(2)

You can manually get back to the first page by going to the "automated" browser and clicking "1".

## Parse the first page of results with Beautiful Soup

Now I'm going to switch to using Beautiful Soup because it's the best program to parse through a lot of HTML.

### Get the table by its `id`

In [18]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
table = soup.find(id='MainContent_gvResult')

In [19]:
# table

Each row of this table is a unique something. I'm not sure what that something is. It might not be a unique case. It might be something else. I'm not going to assume. Anyway, I'd like to transfer this table into a pandas dataframe.

### Create your blank dataframe

In [20]:
hearings = pd.DataFrame(
    columns=[
        'Serial No.',
        'Name',
        'Case #',
        'PFN',
        'CEN',
        'Dept#',
        'Hearing Date',
        'Hearing Time',
        'Hearing Type',
        'Case Type',
        'Defense Atty',
        'DA'
    ])

### Parse the table and put the data into a dataframe

Let's go over each section below manually before running.

In [21]:
# create a simple `page_data` list to store the page data before we make a pandas dataframe
page_data = []
rows = table.find_all('tr')

# we haven't used enumerate() yet but basically that just allows you to index an iterable
for i, row in enumerate(rows):

    # we can skip the first row because that's the header row
    # we can also skip any row greater than index 50 because that has the page numbers
    if (i > 0) and (i <= 50):
    
        # `cells` will get and index all the cells within a row
        cells = row.find_all('td')
        page_data.append({
            'Serial No.' : cells[0].text.strip(),
            'Name' : cells[1].text.strip(), 
            'Case #' : cells[2].text.strip(), 
            'PFN' : cells[3].text.strip(), 
            'CEN' : cells[4].text.strip(), 
            'Dept#' : cells[5].text.strip(), 
            'Hearing Date' : cells[6].text.strip(), 
            'Hearing Time' : cells[7].text.strip(), 
            'Hearing Type' : cells[8].text.strip(), 
            'Case Type' : cells[9].text.strip(), 
            'Defense Atty' : cells[10].text.strip(), 
            'DA' : cells[11].text.strip()
        })
        
# create a dataframe with `page_data`
page_hearing = pd.DataFrame(page_data)
page_hearing

,Serial No.,Name,Case #,PFN,CEN,Dept#,Hearing Date,Hearing Time,Hearing Type,Case Type,Defense Atty,DA
0,1,"FOSTER, JIMMIE LEE",167627,AYO984,1340030,,12/06/2021,9:00AM,Hearing,Felony,,
1,2,"THOMAS, MARKEL",410407,BGN192,8324445,,12/06/2021,9:00AM,Hearing,Misdemeanor/Infractions,,
2,3,"MOORE, MARIA",18-CR-019696B,BMD439,8418630,002,12/06/2021,8:30AM,Further Jury Trial,Felony,Darryl Antonio Stallworth,Alexander Hernandez
3,4,"SALVANT, MARVEL",18-CR-019696A,BMD440,8418631,002,12/06/2021,8:30AM,Further Jury Trial,Felony,Christina Marie Moore,Alexander Hernandez
4,5,"STANDLEY, CURTIS RICHARD",17-CR-027648,BGT167,7831868,003,12/06/2021,8:30AM,Further Jury Trial,Felony,Anne Catherine Beles,Angelina Lisa Clay
5,6,"STANDLEY, CURTIS RICHARD",178554A,BGT167,6294134,003,12/06/2021,8:30AM,Hearing After Revocation of Probation,Felony,David Emmett Klaus,John Norman Creighton
6,7,"WATSON, WILL ANTHONY",17-CR-003246,BLJ096,7803487,005,12/06/2021,8:30AM,Further Jury Trial,Felony,Hugo Ernesto Castillo,Christopher David Cavagnaro
7,8,"CASTRO, FREDY AMAYA",20-CR-011202,BEU230,0503818,006,12/06/2021,8:30AM,Bail Hearing,Felony,Madeline Maria Zepeda,William Matthew Layne
8,9,"CRAWFORD, LAMAR LEEJORDAN",20-CR-010100,DTC648,0501382,006,12/06/2021,8:30AM,Bail Bond Motion,Felony,,William Matthew Layne
9,10,"ELLIS, ANTOINE MARQUIS",18-CR-019861,BCV669,8418862,006,12/06/2021,8:30AM,Motion pursuant to 995PC,Felony,Ernesto H. Castillo,Charles Francis Bisesto


## Append `page_hearing` dataframe to main `hearings` dataframe

In [22]:
hearings = hearings.append(page_hearing).reset_index(drop=True)

## View dataframe

In [23]:
hearings

,Serial No.,Name,Case #,PFN,CEN,Dept#,Hearing Date,Hearing Time,Hearing Type,Case Type,Defense Atty,DA
0,1,"FOSTER, JIMMIE LEE",167627,AYO984,1340030,,12/06/2021,9:00AM,Hearing,Felony,,
1,2,"THOMAS, MARKEL",410407,BGN192,8324445,,12/06/2021,9:00AM,Hearing,Misdemeanor/Infractions,,
2,3,"MOORE, MARIA",18-CR-019696B,BMD439,8418630,002,12/06/2021,8:30AM,Further Jury Trial,Felony,Darryl Antonio Stallworth,Alexander Hernandez
3,4,"SALVANT, MARVEL",18-CR-019696A,BMD440,8418631,002,12/06/2021,8:30AM,Further Jury Trial,Felony,Christina Marie Moore,Alexander Hernandez
4,5,"STANDLEY, CURTIS RICHARD",17-CR-027648,BGT167,7831868,003,12/06/2021,8:30AM,Further Jury Trial,Felony,Anne Catherine Beles,Angelina Lisa Clay
5,6,"STANDLEY, CURTIS RICHARD",178554A,BGT167,6294134,003,12/06/2021,8:30AM,Hearing After Revocation of Probation,Felony,David Emmett Klaus,John Norman Creighton
6,7,"WATSON, WILL ANTHONY",17-CR-003246,BLJ096,7803487,005,12/06/2021,8:30AM,Further Jury Trial,Felony,Hugo Ernesto Castillo,Christopher David Cavagnaro
7,8,"CASTRO, FREDY AMAYA",20-CR-011202,BEU230,0503818,006,12/06/2021,8:30AM,Bail Hearing,Felony,Madeline Maria Zepeda,William Matthew Layne
8,9,"CRAWFORD, LAMAR LEEJORDAN",20-CR-010100,DTC648,0501382,006,12/06/2021,8:30AM,Bail Bond Motion,Felony,,William Matthew Layne
9,10,"ELLIS, ANTOINE MARQUIS",18-CR-019861,BCV669,8418862,006,12/06/2021,8:30AM,Motion pursuant to 995PC,Felony,Ernesto H. Castillo,Charles Francis Bisesto


In [24]:
for n in range(pages_to_check):
    next_button = driver.find_element(By.LINK_TEXT, '>')
    next_button.click()
    time.sleep(2)

KeyboardInterrupt: 

## Addenda 

If I want to search for another date, I can stay on the same page and "clear" the date fields. Then I send send new dates.

In [ ]:
hearing_date_from.clear()
hearing_date_to.clear()

hearing_date_from.send_keys('12/07/2021')
hearing_date_to.send_keys('12/07/2021')

Once you're done using the automated browser, you can close it manually or run the following:

In [ ]:
driver.close()

## Classwork

I'd like you to figure out how to loop through all the pages and collect all the information.